# Load model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ln -s /content/drive/My\ Drive/runs /content/runs

In [ ]:
!pip install ultralytics

from ultralytics import YOLO

In [ ]:
model = YOLO('/content/runs/detect/train10/weights/best.pt')

In [ ]:
# unzip after upload of asl dataset
!unzip asl.zip

# Dataset Retrivial

In [ ]:
import cv2
import os

img_dir = "/content/dataset/images"
label_dir = "/content/dataset/labels"
os.makedirs(img_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)
classes = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "del", "space"]
with open('log.txt', 'w') as l:
  for i in range(len(classes)):
    c = classes[i]
    src_dir = f"/content/asl_alphabet_train/asl_alphabet_train/{c}"
    counter = 0
    file_counter = 0
    for file in os.listdir(src_dir):
    #print(f"[{c} class]:files found: {file_counter} files")
      file_counter += 1;
      img = f'{src_dir}/{file}'
      #print(img)
      results = model.predict(source=f'{src_dir}/{file}')
      # Process results list
      #print(results)
      for result in results:
        boxes = result.boxes
        for box in boxes:
          # Extract coordinates
          #print(box)
          x1, y1, x2, y2 = box.xywhn[0]
          label = f'{i} {x1} {y1} {x2} {y2}'
          #print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
          with open(f'{label_dir}/{file[:-4]}.txt', 'w') as f:
            f.write(label)
          #save the file img in f'{img_dir}/{file}'
          cv2.imwrite(f'{img_dir}/{file}', cv2.imread(img))
          counter += 1
      l.write(f"[{c} class]: {counter} files found among {file_counter} files\n")

# Refactoring of the dataset

In [ ]:
from os.path import exists
import os
import shutil

src_dir = "/content/dataset"
src_img_dir = "/content/dataset/images"
src_label_dir = "/content/dataset/labels"

train_size = 1900
val_size = 200
test_size = 50

classes = ["A", "B", "C", "D", "E,", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "del", "space"]
num_classes = len(classes)
print(num_classes)

os.mkdir("./newdataset")
os.mkdir("./newdataset/train")
os.mkdir("./newdataset/val")
os.mkdir("./newdataset/test")
os.mkdir("./newdataset/train/images")
os.mkdir("./newdataset/train/labels")
os.mkdir("./newdataset/val/images")
os.mkdir("./newdataset/val/labels")
os.mkdir("./newdataset/test/images")
os.mkdir("./newdataset/test/labels")

for c in classes:
	counter = 1
	files_counter = 0
	while counter < 3000:
		filename = f"{c}{counter}.jpg"
		if os.path.exists(os.path.join(src_img_dir, filename)):
			files_counter += 1
			if files_counter <= train_size:
				shutil.copyfile(os.path.join(src_img_dir, filename), os.path.join("./newdataset/train/images", filename))
				shutil.copyfile(os.path.join(src_label_dir, filename.replace(".jpg", ".txt")), os.path.join("./newdataset/train/labels", filename.replace(".jpg", ".txt")))
			elif files_counter <= train_size + val_size:
				shutil.copyfile(os.path.join(src_img_dir, filename), os.path.join("./newdataset/val/images", filename))
				shutil.copyfile(os.path.join(src_label_dir, filename.replace(".jpg", ".txt")), os.path.join("./newdataset/val/labels", filename.replace(".jpg", ".txt")))
			elif files_counter <= train_size + val_size + test_size:
				shutil.copyfile(os.path.join(src_img_dir, filename), os.path.join("./newdataset/test/images", filename))
				shutil.copyfile(os.path.join(src_label_dir, filename.replace(".jpg", ".txt")), os.path.join("./newdataset/test/labels", filename.replace(".jpg", ".txt")))
			else:
				break
		print(f"[Class {c}] {files_counter}/2150 files; {files_counter*100/2150}%")
		counter += 1

# Util: show yolo

In [ ]:
import cv2
import numpy

def show_yolo(image_path, label_path):
    # Read the image
    image = cv2.imread(image_path)

    # Read the label
    with open(label_path, "r") as f:
        label = f.read().split()

    # Get the bounding box coordinates
    x_center = float(label[1]) * 640
    y_center = float(label[2]) * 640
    width = float(label[3]) * 640
    height = float(label[4]) * 640

    # Calculate the bounding box coordinates
    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)

    # Draw the bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Show the image
    cv2.imshow("Image with bounding box", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()